In [1]:
# Parameters
mongo_host = "localhost"
mongo_port = 27017
db_name = "admin"
collection_name = "emaildatafeeder"
last_timestamp_collection_name = "lasttimestamp"
cohort_all_collection = "cohort_id"


In [2]:
!pip install isodate


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import papermill as pm
import dateutil.parser
import dateutil
import pandas as pd
import numpy as np
import pymongo
from config import params  # Import parameters from the config file

# Extract parameters for MongoDB connection
mongo_host = params['mongo_host']
mongo_port = params['mongo_port']
db_name = params['db_name']
collection_name = params['collection_name']
last_timestamp_collection_name = params['last_timestamp_collection_name']

# Establish MongoDB connection
client = pymongo.MongoClient(f"mongodb://{mongo_host}:{mongo_port}")
db = client[db_name]

#class processing_data:
#############Sub Functions ################

# Define the update_last_processed_timestamp_to_db function
def update_last_processed_timestamp_to_db(timestamp):
    print('timestamp:', timestamp, "collection:", last_timestamp_collection_name)
    last_timestamp_collection = db[last_timestamp_collection_name].update_one({}, {"$set": {"last_processed_timestamp": timestamp}}, upsert=True)
    print('update output', last_timestamp_collection)
    cursor = db[last_timestamp_collection_name].find({})
    documents = list(cursor)
    cursor.close()

def process_since_last_timestamp(last_timestamp):
    vtime = {'Timestamp': {'$gt': last_timestamp}}
    cursor = db[collection_name].find(vtime)
    documents = list(cursor)
    
    ## karu for time being neeed to check it out cursor.close()
    
    cdc_data = pd.DataFrame(documents)

    if len(cdc_data) == 0:
        print('No records processed...')
    else:
        max_timestamp = cdc_data['Timestamp'].max()
        update_last_processed_timestamp_to_db(max_timestamp)

    return cdc_data

def process_all_records():
    cursor = db[collection_name].find({})
    documents = list(cursor)
    cursor.close()

    cdc_data = pd.DataFrame(documents)

    cdc_data['Timestamp'] = pd.to_datetime(cdc_data['Timestamp'])

    if len(cdc_data) == 0:
        print('No records processed...')
    else:
        max_timestamp = cdc_data['Timestamp'].max()
        print('Timestamp of last processed data:', max_timestamp)
        update_last_processed_timestamp_to_db(max_timestamp)

    return cdc_data

def get_last_processed_timestamp():
    update_last_processed_timestamp_to_db(0)

    last_timestamp_collection = db[last_timestamp_collection_name]
    last_processed_timestamp = last_timestamp_collection.find_one({}).get('last_processed_timestamp')

    print('last_processed_timestamp:', last_processed_timestamp)

    if last_processed_timestamp == 0 or last_processed_timestamp is None or pd.isnull(last_processed_timestamp):
        print('Processing all records')
        cdc_data = process_all_records()
    else:
        if last_processed_timestamp == 0:
            print('Processing all records since it is 0')
            cdc_data = process_all_records()
        else:
            print('Processing cdc records:')
            print(last_processed_timestamp)
            cdc_data = process_since_last_timestamp(last_processed_timestamp)

    client.close()

    return cdc_data

# Call the get_last_processed_timestamp function
fcdc_data = get_last_processed_timestamp()

# Print the resulting DataFrame
print(fcdc_data)

timestamp: 0 collection: lasttimestamp
update output <pymongo.results.UpdateResult object at 0x000002A2A53165C0>
last_processed_timestamp: 0
Processing all records
Timestamp of last processed data: 2023-11-27 15:03:00
timestamp: 2023-11-27 15:03:00 collection: lasttimestamp
update output <pymongo.results.UpdateResult object at 0x000002A2B0AFD780>
                           _id                        Emailid    Event  \
0     6577eaf3435dd02306e46857           mayscott@example.net   opened   
1     6577eaf3435dd02306e46858       johnsondavid@example.net   opened   
2     6577eaf3435dd02306e46859         vwilkinson@example.com   opened   
3     6577eaf3435dd02306e4685a       millermelvin@example.net   opened   
4     6577eaf3435dd02306e4685b     jillianfuentes@example.net   opened   
...                        ...                            ...      ...   
5013  65794aab86e1c204df55258b            pturner@example.com  clicked   
5014  65794aab86e1c204df55258c          abuchanan@example.c

In [4]:
fcdc_data

,_id,Emailid,Event,Timestamp,Day,Holiday,Country
0,6577eaf3435dd02306e46857,mayscott@example.net,opened,2022-11-25 16:16:00,Friday,True,India
1,6577eaf3435dd02306e46858,johnsondavid@example.net,opened,2023-10-02 02:17:00,Tuesday,True,India
2,6577eaf3435dd02306e46859,vwilkinson@example.com,opened,2023-08-01 02:06:00,Monday,False,India
3,6577eaf3435dd02306e4685a,millermelvin@example.net,opened,2023-08-21 06:16:00,Monday,False,India
4,6577eaf3435dd02306e4685b,jillianfuentes@example.net,opened,2023-10-17 14:03:00,Saturday,True,India
...,...,...,...,...,...,...,...
5013,65794aab86e1c204df55258b,pturner@example.com,clicked,2022-12-16 18:50:00,Tuesday,False,India
5014,65794aab86e1c204df55258c,abuchanan@example.com,clicked,2023-09-08 09:57:00,Sunday,True,India
5015,65794aab86e1c204df55258d,bryantchristopher@example.com,clicked,2023-04-05 21:09:00,Tuesday,True,India
5016,65794aab86e1c204df55258e,patrickpaul@example.net,clicked,2023-09-25 14:43:00,Saturday,True,India


In [5]:
import pandas as pd
import random

# Your existing functions remain unchanged
def assign_numeric_label(hour):
    if 0 <= hour < 6:
        return '1'
    elif 6 <= hour < 12:
        return '2'
    elif 12 <= hour < 18:
        return '3'
    else:
        return '4'

def find_max_values(df):
    label_cols = ['1', '2', '3', '4']
    
    for col in label_cols:
        if col not in df.columns:
            df[col] = 0  
    
    df['recommended_hr'] = df[label_cols].max(axis=1)
    df['hr_id'] = df[label_cols].idxmax(axis=1)
    max_counts = df[label_cols].eq(df['recommended_hr'], axis=0).sum(axis=1)
    multiple_max = df[max_counts > 1].index
    random_column = random.choice(label_cols)
    return df, multiple_max, random_column

def process_emaildata(vgroup):
    vgroup['Timestamp'] = pd.to_datetime(vgroup['Timestamp'])
    vgroup['DayOfWeek'] = vgroup['Timestamp'].dt.dayofweek 
    vgroup['Timestamp'] = vgroup['Timestamp'].dt.hour.apply(assign_numeric_label)
    
    weekdays = vgroup[vgroup['DayOfWeek'] < 5]
    weekends = vgroup[vgroup['DayOfWeek'] >= 5]
    
    vgroupings = ['Event', 'Holiday','Timestamp']
    weekday_final = pd.concat([weekdays.groupby(['Emailid', col]).size().unstack(fill_value=0) for col in vgroupings], axis=1)
    weekend_final = pd.concat([weekends.groupby(['Emailid', col]).size().unstack(fill_value=0) for col in vgroupings], axis=1)
    
    weekday_final, weekday_multiple_max, random_col_weekday = find_max_values(weekday_final)
    weekend_final, weekend_multiple_max, random_col_weekend = find_max_values(weekend_final)
    
    weekday_final.rename(columns={'recommended_hr': 'w_rec_hr'}, inplace=True)
    weekend_final.rename(columns={'recommended_hr': 'we_rec_hr'}, inplace=True)
    
    return weekday_final, weekend_final, weekday_multiple_max, weekend_multiple_max, random_col_weekday, random_col_weekend

In [6]:
# grouped = df.groupby('Emailid')
grouped = fcdc_data.groupby('Emailid')
weekday_email_data = []
weekend_email_data = []
weekday_multiple_max_values = []
weekend_multiple_max_values = []
weekday_random_cols = []
weekend_random_cols = []

for vname, vgroup in grouped:
    (weekday_processed_data, weekend_processed_data,
     weekday_multiple_max, weekend_multiple_max,
     random_col_weekday, random_col_weekend) = process_emaildata(vgroup)
    
    weekday_email_data.append(weekday_processed_data)
    weekend_email_data.append(weekend_processed_data)
    weekday_multiple_max_values.append(weekday_multiple_max)
    weekend_multiple_max_values.append(weekend_multiple_max)
    weekday_random_cols.append(random_col_weekday)
    weekend_random_cols.append(random_col_weekend)

weekday_result = pd.concat(weekday_email_data)
weekend_result = pd.concat(weekend_email_data)

weekday_result.fillna(0, inplace=True)
weekend_result.fillna(0, inplace=True)

combined_result = pd.concat([weekday_result, weekend_result], axis=1)


combined_result['weekday_cohort_id'] = np.where(combined_result['w_rec_hr'] >= combined_result['we_rec_hr'],combined_result['w_rec_hr'],0)

combined_result['weekend_cohort_id'] = np.where(combined_result['we_rec_hr'] > combined_result['w_rec_hr'],combined_result['we_rec_hr'],0)


equal_scores = combined_result['w_rec_hr'] == combined_result['we_rec_hr']
indices = equal_scores[equal_scores].index

for index in indices:
    if combined_result.loc[index, 'weekday_cohort_id'] == combined_result.loc[index, 'weekend_cohort_id']:
        random_column = np.random.choice(['weekday_cohort_id', 'weekend_cohort_id'])
        value = combined_result.loc[index, random_column]
        combined_result.loc[index, 'weekday_cohort_id'] = value
        combined_result.loc[index, 'weekend_cohort_id'] = 0 if random_column == 'weekday_cohort_id' else value
combined_result.reset_index(inplace=True)
print("Combined Data with 'weekday_cohort_id' and 'weekend_cohort_id' columns:")
print(combined_result)

Combined Data with 'weekday_cohort_id' and 'weekend_cohort_id' columns:
                    Emailid  clicked  opened False  True   1   2   3  4  \
0     abuchanan@example.com       18      23    24    17   8  12  12  9   
1     allison04@example.org       19      17    18    18   8   6  15  7   
2      alyssa97@example.com       14      27    22    19  10  13  13  5   
3      andrea82@example.com       20      17    19    18  11   9  10  7   
4   andrewhicks@example.org       16      10    10    16  11   3   6  6   
..                      ...      ...     ...   ...   ...  ..  ..  .. ..   
95    william58@example.net       13      11    11    13  10   6   6  2   
96     wsweeney@example.net       21      22    26    17   8  14  17  4   
97        xbush@example.net       14      12    10    16   8   8   6  4   
98       xevans@example.com       21      12    22    11  11  11   8  3   
99      xthomas@example.org       13      10    11    12   4   2  11  6   

    w_rec_hr  ... False  Tr

In [7]:
combined_result

,Emailid,clicked,opened,False,True,1,2,3,4,w_rec_hr,...,False,True,2,3,4,1,we_rec_hr,hr_id,weekday_cohort_id,weekend_cohort_id
0,abuchanan@example.com,18,23,24,17,8,12,12,9,12,...,8,8.0,5,5,6,0,6,4,12,0
1,allison04@example.org,19,17,18,18,8,6,15,7,15,...,7,4.0,3,6,0,2,6,3,15,0
2,alyssa97@example.com,14,27,22,19,10,13,13,5,13,...,14,7.0,3,8,4,6,8,3,13,0
3,andrea82@example.com,20,17,19,18,11,9,10,7,11,...,9,7.0,7,2,5,2,7,2,11,0
4,andrewhicks@example.org,16,10,10,16,11,3,6,6,11,...,13,3.0,4,4,4,4,4,1,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,william58@example.net,13,11,11,13,10,6,6,2,10,...,7,6.0,5,5,2,1,5,2,10,0
96,wsweeney@example.net,21,22,26,17,8,14,17,4,17,...,8,10.0,2,10,3,3,10,3,17,0
97,xbush@example.net,14,12,10,16,8,8,6,4,8,...,1,2.0,1,1,0,1,1,1,8,0
98,xevans@example.com,21,12,22,11,11,11,8,3,11,...,4,10.0,7,4,3,0,7,2,11,0


In [8]:
combined_result.columns = combined_result.columns.astype(str)

# Create MongoDB client and connect to the database
mongo_uri = f"mongodb://{mongo_host}:{mongo_port}"
client = pymongo.MongoClient(mongo_uri)
db = client[db_name]
collection = db[cohort_all_collection]

data_to_insert = combined_result.to_dict(orient="records")

data_to_insert = [{str(key): value for key, value in doc.items()} for doc in data_to_insert]

collection.insert_many(data_to_insert)

client.close()

print("Data has been successfully saved to the MongoDB database.")


Data has been successfully saved to the MongoDB database.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7768\1069721817.py:9: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data_to_insert = combined_result.to_dict(orient="records")


In [9]:
# combined_result.to_csv('cohort_all_outpute_testing1.csv')